In [ ]:
import pandas as pd
import wget
import re

In [ ]:
## Download the data
DSET_URL = {
    "ATP": "https://webs.iiitd.edu.in/raghava/ccpdb/datasets/ATP.txt",
    "ADP": "https://webs.iiitd.edu.in/raghava/ccpdb/datasets/ADP.txt",
    "GTP": "https://webs.iiitd.edu.in/raghava/ccpdb/datasets/GTP.txt",
    "GDP": "https://webs.iiitd.edu.in/raghava/ccpdb/datasets/GDP.txt",
    "NAD": "https://webs.iiitd.edu.in/raghava/ccpdb/datasets/NAD.txt",
    "FAD": "https://webs.iiitd.edu.in/raghava/ccpdb/datasets/FAD.txt",
    "FMN": "https://webs.iiitd.edu.in/raghava/ccpdb/datasets/FMN.txt",
    "UDP": "https://webs.iiitd.edu.in/raghava/ccpdb/datasets/UDP.txt",
}

for k, v in DSET_URL.items():
    print(f"Downloading: {k}") 
    wget.download(v)

In [ ]:
from Bio import SeqIO

def parse_dset(filename: str, write: bool = False) -> pd.DataFrame:
    with open(filename) as fasta_file:  # Will close handle cleanly
        identifiers = []
        chains = []
        sequences = []
        interactions = []
        lengths = []
        for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
            identifiers.append(seq_record.id[:-1])
            chains.append(seq_record.id[-1])
            lengths.append(len(seq_record.seq)/2)

            parsed_sequence_and_interactions = re.split(';', re.sub("\+|-", lambda match: ';' + match.group(), str(seq_record.seq), count=1), maxsplit=1)#str(seq_record.seq).split("-", maxsplit=1)
            sequences.append(parsed_sequence_and_interactions[0])
            interactions.append(parsed_sequence_and_interactions[1])

    data = zip(identifiers, chains, sequences, interactions,lengths)
    df = pd.DataFrame.from_dict(data)
    df.columns = ["PDB", "chain", "sequence", "interacting_residues", "length"]
    df["interactor"] = filename.split(".")[0]
    
    if write:
        df.to_csv(filename.split(".")[0] + ".csv")
    return df

In [ ]:
## Iterate over datasets
dataset_list = [parse_dset(k + ".txt", write=True) for k in DSET_URL.keys()]

In [ ]:
df = pd.concat(dataset_list)
df.reset_index(inplace=True, drop=True)
df.to_csv("PROTEINS_NUCLEOTIDES.csv")

In [ ]:
df